In [82]:
import requests
from dotenv import load_dotenv
import os
from transformers import BartTokenizer, AutoTokenizer


In [83]:
load_dotenv()

True

In [84]:
HF_TOKEN = os.getenv("HUGGING_FACE_API_TOKEN")
#API_URL = "https://api-inference.huggingface.co/models/sshleifer/distilbart-cnn-12-6"

API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
#tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")


In [85]:
def chunk_text(text, max_tokens):
    tokens = tokenizer(text, truncation=False, return_tensors="pt")['input_ids'][0]
    for i in range(0, len(tokens), max_tokens):
        yield tokenizer.decode(tokens[i:i + max_tokens], skip_special_tokens=True)

In [86]:
def query(payload):
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    response = requests.post(API_URL, headers=headers, json=payload)

    return response.json()

In [87]:
long_text = "Durchführung einer umfassenden Kosinus-Ähnlichkeitsanalyse untersucht dieser Artikel die Abstimmungsverhalten von Bundestagsabgeordneten (MdBs) über vier Legislaturperioden in der deutschen politischen Geschichte. Der Schwerpunkt liegt auf Kabinett I unter Willy Brandt (SPD & FDP), Kabinett II unter Helmut Kohl (CDU & FDP), sowie Kabinett I und IV unter Angela Merkel (CDU & SPD). Das Hauptziel besteht darin, die komplexen Dynamiken politischer Allianzen zu entschlüsseln. Die Methodik umfasst die Berechnung der Kosinus-Ähnlichkeit für jedes MdB-Paar basierend auf ihren gemeinsamen Abstimmungen. Die zugrunde liegende Hypothese besagt, dass Regierungskoalitionswechsel die Muster von Kooperation und Opposition unter den MdBs signifikant beeinflussen. Um die Analyse effizienter zu gestalten und das Wesentliche der Abstimmungsverhalten der MdBs zu erfassen, wurde der Datensatz auf die ersten und letzten drei Monate jeder Legislaturperiode reduziert. Dieser fokussierte Ansatz destilliert die Kerndynamiken effizient, während die Komplexität gemindert wird. Durch die Konzentration auf die Anfangs- und Endphasen, die durch bedeutende politische Debatten und gesetzgeberische Entscheidungen geprägt sind, wird ein repräsentatives Ergebnis der Analyse gewährleistet. Diese gezielte Untersuchung ermöglicht es, subtile Veränderungen und Ausrichtungen in den Abstimmungsmustern zu identifizieren, was zu einem umfassenden Verständnis der breiteren Legislaturperioden beiträgt. Die Ergebnisse unterstreichen das fein abgestimmte Zusammenspiel zwischen politischen Zugehörigkeiten und politischer Ausrichtung und liefern wertvolle Einblicke in die dynamische Natur politischer Allianzen im Deutschen Bundestag. Besonders hervorzuheben ist eine aufschlussreiche Beobachtung: Mitglieder der Freien Demokratischen Partei (FDP) zeigten eine erhöhte Abstimmungsähnlichkeit mit der Christlich Demokratischen Union (CDU), wenn beide Parteien Teil derselben Regierungskoalition waren. Diese Beobachtung stützt die zentrale Hypothese, dass Regierungskoalitionswechsel tatsächlich erheblichen Einfluss auf die Kooperations- und Divergenzmuster unter den MdBs haben. Die umfassende Kosinus-Ähnlichkeitsanalyse der Abstimmungsmuster von Bundestagsabgeordneten über die genannten historischen Perioden unterstützt die Annahme, dass politische Allianzen ihre kooperativen Dynamiken maßgeblich prägen. Durch die detaillierte Untersuchung spezifischer Kabinette unter Führung von Schlüsselpersönlichkeiten wie Willy Brandt, Helmut Kohl und Angela Merkel beleuchtet dieser Artikel nicht nur die breite Entwicklung politischer Zugehörigkeiten, sondern wirft auch ein Licht auf die zeitlichen Dynamiken innerhalb jeder Legislaturperiode. Abschließend bietet dieser Artikel eine wertvolle Untersuchung des Zusammenspiels zwischen politischen Allianzen und Abstimmungsmustern im Deutschen Bundestag. Er liefert Einsichten, die unser Verständnis der dynamischen Natur politischer Prozesse vertiefen. Die beobachteten Muster validieren nicht nur die ursprüngliche Hypothese, sondern präsentieren auch eine überzeugende Erzählung darüber, wie Regierungskoalitionswechsel die Kooperationslandschaft unter MdBs in verschiedenen Legislaturperioden der deutschen politischen Geschichte prägen."

In [88]:
chunks = list(chunk_text(long_text, max_tokens=900))  # Adjust max_tokens if necessary

Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors


In [89]:

results = []
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}...")
    payload = {"inputs": chunk,
               "parameters": {
                   "truncation": True,
                   "max_length": 1024  # Adjust based on the model's limit
               }}
    result = query(payload)
    results.append(result)

Processing chunk 1/2...
Processing chunk 2/2...


In [91]:
results

[{'error': 'Model too busy, unable to get response in less than 60 second(s)'},
 {'error': 'Internal Server Error'}]

In [59]:
combined_summary = " ".join(
    item["summary_text"]
    for result in results  # Iterate over the outer list
    for item in result  # Iterate over the inner lists
    if "summary_text" in item  # Check if 'summary_text' exists
)
combined_summary

'Durchführung einer umfassenden Kosinus-Ähnlichkeitsanalyse untersucht dieser Artikel die Abstimmungsverhalten von Bundestagsabgeordneten. Das Hauptziel besteht darin, die komplexen Dynamiken politischer Allianzen zu entschlüsseln. Artikel beleuchtet nicht nur die breite Entwicklung politischer Zugehörigkeiten, sondern ein Licht auf die zeitlichen Dynamiken innerhalb jeder Legislaturperiode. Er liefert Einsichten, die unser Verständnis der dynamischen Natur politischer Prozesse vertiefen.'

In [57]:
combined_summary

''

In [100]:
import json

json_object = '{"key1": "value1", "key2": "value2", "key3": "value3"}'
parsed_object = json.loads(json_object)  

second_key = list(parsed_object.keys())[1]
second_value = parsed_object[second_key]
print(second_value)

value2
